# CS181 Practical I

In [10]:
import numpy as np
import matplotlib.pyplot as plt

## Load the data files into local memory from the GCP bucket.

In [23]:
!gsutil cp gs://cs181_practical_data/Xtrain_amp.npy ./
!gsutil cp gs://cs181_practical_data/ytrain_amp.npy ./

!gsutil cp gs://cs181_practical_data/Xtest_amp.npy ./
!gsutil cp gs://cs181_practical_data/ytest_amp.npy ./

!gsutil cp gs://cs181_practical_data/Xtrain_mel.npy ./
!gsutil cp gs://cs181_practical_data/ytrain_mel.npy ./

!gsutil cp gs://cs181_practical_data/Xtest_mel.npy ./
!gsutil cp gs://cs181_practical_data/ytest_mel.npy ./

Copying gs://cs181_practical_data/Xtrain_amp.npy...
If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

/ [1 files][934.2 MiB/934.2 MiB]   35.9 MiB/s                                   
Operation completed over 1 objects/934.2 MiB.                                    
Copying gs://cs181_practical_data/ytrain_amp.npy...
/ [1 files][ 43.5 KiB/ 43.5 KiB]                                                
Operation completed over 1 objects/43.5 KiB.                                     
Copying gs://cs181_practical_data/Xtest_amp.npy...
If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .

## Imports

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import librosa

from torch import nn
from torchsummary import summary




### Load raw amplitude data as a numpy object.

In [11]:
# Load train data

X_amp_train = np.load("Xtrain_amp.npy")
y_amp_train = np.load("ytrain_amp.npy")

In [27]:
# Load test data

X_amp_test = np.load("Xtest_amp.npy")
y_amp_test = np.load("ytest_amp.npy")

### Load Mel spectrogram data as a numpy object.

In [15]:
# Load train data

X_mel_train = np.load("Xtrain_mel.npy")
y_mel_train = np.load("ytrain_mel.npy")

In [16]:
X_mel_train.shape

(5553, 128, 87)

In [17]:
# Flatten X_mel_train's spectrogram features
X_mel_train_flat = X_mel_train.reshape(X_mel_train.shape[0], -1)
X_mel_train_flat.shape

(5553, 11136)

In [18]:
# Load test data

X_mel_test = np.load("Xtest_mel.npy")
y_mel_test = np.load("ytest_mel.npy")

In [19]:
X_mel_test.shape

(2197, 128, 87)

In [47]:
# Flatten X_mel_test's spectrogram features
X_mel_test_flat = X_mel_test.reshape(X_mel_test.shape[0], -1)
X_mel_test_flat.shape

(2197, 11136)

## Metrics

In [38]:
def get_metrics(model, is_raw=True):
    # we can also get TP, TN, FP, FN from this
    overall_train_acc = -1
    overall_test_acc = -1
    train_matrix = -1
    test_matrix = -1
    other_metrics_train = -1
    other_metrics_test = -1

    if is_raw:
        overall_train_acc = model.score(X_amp_train, y_amp_train)
        overall_test_acc = model.score(X_amp_test, y_amp_test)

        y_pred_train = model.predict(X_amp_train)
        y_pred_test = model.predict(X_amp_test)

        train_matrix = confusion_matrix(y_amp_train, y_pred_train)
        train_matrix = train_matrix.diagonal()/train_matrix.sum(axis=1)
        test_matrix = confusion_matrix(y_amp_test, y_pred_test)
        test_matrix = test_matrix.diagonal()/test_matrix.sum(axis=1)

        other_metrics_train = precision_recall_fscore_support(y_amp_train, y_pred_train)
        other_metrics_test = precision_recall_fscore_support(y_amp_test, y_pred_test)

    else:
        overall_train_acc = model.score(X_mel_train_flat, y_mel_train)
        overall_test_acc = model.score(X_mel_test_flat, y_mel_test)

        y_pred_train = model.predict(X_mel_train_flat)
        y_pred_test = model.predict(X_mel_test_flat)

        train_matrix = confusion_matrix(y_mel_train, y_pred_train)
        train_matrix = train_matrix.diagonal()/train_matrix.sum(axis=1)
        test_matrix = confusion_matrix(y_mel_test, y_pred_test)
        test_matrix = test_matrix.diagonal()/test_matrix.sum(axis=1)

        other_metrics_train = precision_recall_fscore_support(y_mel_train, y_pred_train)
        other_metrics_test = precision_recall_fscore_support(y_mel_test, y_pred_test)

    print('overall accuracy (train): ', overall_train_acc)
    print('overall accuracy (test): ', overall_test_acc)
    print('per-class accuracies (train): ', train_matrix)
    print('per-class accuracies (test): ', test_matrix)
    print('per-class precision, recall, f1, support (train): ', other_metrics_train)
    print('per-class precision, recall, f1, support (test): ', other_metrics_test)


In [45]:
'''
Calculate overall accuracy, and per-class accuracy
'''
def calc_metrics(model, is_raw_amp=True):
    if is_raw_amp:
        X_test = X_amp_test
        y_test = y_amp_test
        X_train = X_amp_train
        y_train = y_amp_train
    else:
        X_test = X_mel_test_flat
        y_test = y_mel_test
        X_train = X_mel_train_flat
        y_train = y_mel_train

    # Calculate overall accuracy
    test_acc_overall = model.score(X_test, y_test)
    train_acc_overall = model.score(X_train, y_train)

    # Calculate per-class accuracy
    y_pred_test = model.predict(X_test)
    y_pred_train = model.predict(X_train)

   # Calculate accuracy for each dataset using confusion matrix
    cm_test = confusion_matrix(y_test, y_pred_test)
    cm_train = confusion_matrix(y_train, y_pred_train)

    test_acc_per_class = cm_test.diagonal()/cm_test.sum(axis=1)
    train_acc_per_class = cm_train.diagonal()/cm_train.sum(axis=1)

    classes = ['air_conditioner', 'car_horn', 'children_playing', 'dog_bark', 'drilling', 'engine_idling', 'gun_shot', 'jackhammer', 'siren', 'street_music']

    # Display metrics using classes as labels

    print("Overall accuracy on test set: {0:.4f}".format(test_acc_overall))
    print("Overall accuracy on train set: {0:.4f}".format(train_acc_overall))

    print("\nPer-class accuracy on test set:")
    for i in range(len(classes)):
        print("{0}: {1:.4f}".format(classes[i], test_acc_per_class[i]))

    print("\nPer-class accuracy on train set:")
    for i in range(len(classes)):
        print("{0}: {1:.4f}".format(classes[i], train_acc_per_class[i]))
        

## Part A: Feature Engineering and Baseline Models

### Raw Amplitude features

In [43]:
classifier_raw = LogisticRegression(penalty='l2', tol = 10**-3, max_iter=1000)
classifier_raw.fit(X_amp_train, y_amp_train)

LogisticRegression(max_iter=1000, tol=0.001)

In [42]:
calc_metrics(classifier_raw, is_raw_amp=True)

Overall accuracy on test set: 0.1789
Overall accuracy on train set: 0.9773

Per-class accuracy on test set:
air_conditioner: 0.2867
car_horn: 0.0000
children_playing: 0.3445
dog_bark: 0.1223
drilling: 0.0114
engine_idling: 0.3068
gun_shot: 0.0667
jackhammer: 0.0424
siren: 0.1398
street_music: 0.1567

Per-class accuracy on train set:
air_conditioner: 0.9929
car_horn: 0.9898
children_playing: 0.9813
dog_bark: 0.9503
drilling: 0.9901
engine_idling: 0.9764
gun_shot: 0.9759
jackhammer: 0.9954
siren: 0.9416
street_music: 0.9814


In [39]:
get_metrics(classifier_raw, is_raw=True)

overall accuracy (train):  0.9773095623987034
overall accuracy (test):  0.17888029130632682
per-class accuracies (train):  [0.99285714 0.98984772 0.98132184 0.95028681 0.99011532 0.97642164
 0.97590361 0.99544073 0.94161677 0.98142857]
per-class accuracies (test):  [0.28666667 0.         0.34448161 0.12227074 0.01136364 0.30681818
 0.06666667 0.04237288 0.13983051 0.15666667]
per-class precision, recall, f1, support (train):  (array([0.94173442, 1.        , 0.90824468, 1.        , 1.        ,
       0.99858156, 1.        , 1.        , 0.97975078, 1.        ]), array([0.99285714, 0.98984772, 0.98132184, 0.95028681, 0.99011532,
       0.97642164, 0.97590361, 0.99544073, 0.94161677, 0.98142857]), array([0.96662031, 0.99489796, 0.94337017, 0.9745098 , 0.99503311,
       0.98737728, 0.98780488, 0.99771516, 0.96030534, 0.99062725]), array([700, 197, 696, 523, 607, 721,  83, 658, 668, 700]))
per-class precision, recall, f1, support (test):  (array([0.16538462, 0.        , 0.18070175, 0.168674

### Mel Spectogram Features

In [22]:
classifier_mel = LogisticRegression(random_state=0, tol = 10**-4, max_iter=1000)
classifier_mel.fit(X_mel_train_flat, y_mel_train)

/Users/adammo/Desktop/cs181-env/cs181-s23-homeworks/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, random_state=0)

In [48]:
calc_metrics(classifier_mel, is_raw_amp=False)

Overall accuracy on test set: 0.3669
Overall accuracy on train set: 0.9755

Per-class accuracy on test set:
air_conditioner: 0.2867
car_horn: 0.3333
children_playing: 0.2308
dog_bark: 0.1921
drilling: 0.4583
engine_idling: 0.4053
gun_shot: 0.7000
jackhammer: 0.4619
siren: 0.7034
street_music: 0.2333

Per-class accuracy on train set:
air_conditioner: 0.9771
car_horn: 0.9898
children_playing: 0.9598
dog_bark: 0.9446
drilling: 0.9885
engine_idling: 0.9598
gun_shot: 1.0000
jackhammer: 0.9954
siren: 0.9955
street_music: 0.9729
